In [2]:
import gc
import wandb
import pandas as pd
import torch
import torch.cuda

c:\users\이육샛별\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.getcwd()

'C:\\Users\\이육샛별\\Franklin\\KoFairytaleGenerator\\Model'

In [4]:
from pathlib import Path
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments

from transformers import TextDataset,DataCollatorForLanguageModeling

In [19]:
!pip install wandb
!pip install ipywidgets
!pip install scikit-learn
!pip install whatlies[all]

In [20]:
torch.__version__
torch.cuda.is_available()
!nvidia-smi

Fri Sep  2 16:29:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:02:00.0 Off |                  Off |
| 30%   35C    P8    15W / 230W |   2723MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
def read_txt(file_path):
    with open(file_path, 'r', encoding ="UTF-8") as f:
    #with open(file_path, 'r') as f:
        fr = f.read()
    return fr

def read_csv(file_path):
    fr = pd.read_csv(file_path, encoding = "utf-8")
    return fr

In [8]:
dataset = read_csv('../Dataset/dataset_0902.csv')
dataset = dataset.drop(columns=['Unnamed: 0'])
dataset

,동화
0,"""나는 알쏭달쏭 미술관 관장 '다다'란다. 이곳은 신기하고 괴상한 작품으로 가득하지..."
1,방정환은 어려서부터 책을 참 좋아했어요. 다섯 살 때는 할아버지한테 천재문을 배웠죠...
2,"텔레비전도 재미있는 그림책도 없었던 옛날의 어린이들은 무엇을 하고 놀았을까요? ""옛..."
3,"""따뜻한 봄바람이 살랑살랑 불고 겨울에 움츠렸던 몸이 움찔움찔. 얘들아 나랑 놀자 ..."
4,우리 할아버지입니다. 어려서부터 나는 할아버지와 한 방을 썼어요. 할아버지는 늘 신...
...,...
772,핸드폰 게임 앱을 누르자 선물상자가 나왔다. 선물상자가 열리면서 수많은 옷들과 폭죽...
773,"그럭저럭 사이좋은 부부가 있었어요. 둘은 같이 식탁을 차렸고, 저녁을 먹은 뒤에는 ..."
774,비가 개었다. 동시에 저 편 들판 건너 숲 뒤에는 둥그렇게 무지개가 뻗쳤다. 오묘...
775,여름 장이란 애시당초에 글러서 해는 아직 중천에 있건만 장판은 벌써 쓸쓸하고 더운 ...


In [13]:
swapped_dataset = read_csv('../Dataset/swapped_data_0902.csv')
swapped_dataset = swapped_dataset.drop(columns=['Unnamed: 0'])
swapped_dataset

,동화
0,"""나는 알쏭달쏭 미술관 관장 '<인물0>'란다. 이곳은 신기하고 괴상한 작품으로 가..."
1,<인물1>은 어려서부터 책을 참 좋아했어요. 다섯 살 때는 할머니한테 천재문을 배웠...
2,"텔레비전도 재미있는 그림책도 없었던 옛날의 어린이들은 무엇을 하고 놀았을까요? ""옛..."
3,"""따뜻한 봄바람이 살랑살랑 불고 겨울에 움츠렸던 몸이 움찔움찔. 얘들아 나랑 놀자 ..."
4,우리 할아버지입니다. 어려서부터 나는 할머니와 한 방을 썼어요. 할머니는 늘 신문을...
...,...
772,핸드폰 게임 앱을 <인물153>자 선물상자가 나왔다. 선물상자가 열리면서 수많은 옷...
773,"그럭저럭 사이좋은 부부가 있었어요. 둘은 같이 식탁을 차렸고, 저녁을 먹은 뒤에는 ..."
774,비가 개었다. 동시에 저 편 들판 건너 숲 뒤에는 둥그렇게 무지개가 뻗쳤다. 오묘...
775,여름 장이란 애시당초에 글러서 해는 아직 중천에 있건만 장판은 벌써 쓸쓸하고 더운 ...


In [14]:
concat_dataset = dataset + swapped_dataset
concat_dataset['동화'][0]

'"나는 알쏭달쏭 미술관 관장 \'다다\'란다. 이곳은 신기하고 괴상한 작품으로 가득하지. 재미있는 미술 작품들이 가득한 알쏭달쏭 미술관에 온 걸 환영합니다." "이쪽에 있는 변기를 볼래? 화장실에서나 볼 수 있는 변기가 미술 작품이라면 믿겠니?" "네? 변기가 미술 작품이라구요?" "너희들 모나리자란 그림을 알고 있지? 레오나르도 다빈치가 그린 그림이야. 그런데 뒤샹이라는 화가는 모나리자의 얼굴에 수염만 그리고 전시를 했단다. 그래서 사람들은 깜짝 놀랐지." "네? 유명한 작품에 낙서를 한 게 미술 작품이라구요?" "뒤샹은 장난꾸러기였나봐. 그럼 이거는 뭘 그린 건지 한번 맞춰볼래? 여러 색깔의 물감을 뿌리고 붓고 흘린 거란다. 곳곳에 커다란 물감 얼룩도 있지." "폴록이라는 화가는 이게 연보라빛 안개라는구나. 그런데 미술관 관장인 내가 봐도 신나게 물감을 뿌려 놓은 것 같거든 혹시 너희들 눈에는 안개가 보이니?" "글쎄요? 보이는 것도 같고 아닌 것 같기도 하고.." "그렇게 볼록은 커다란 종류의 물감을 쏟아 부었어. 그리고 붓을 휘젓고 물감을 튀겼지. 가끔은 물감 대신 모래를 톡톡 뿌리기도 했단다. 진짜 신났겠지?" "물감을 가지고 장난치는 것 같아..." "화가 에른스트는 물건의 종이를 대고 문지르면서 작품을 만들었어. 어떤 물건이냐에 따라 무늬가 달라지지!" "에이 이건 나도 그릴 수 있을 것 같아." "종이에 물감을 쭉 짜놓고 접었다가 펴면 양쪽에 똑같은 그림이 나타나지 이걸 \'데칼코마니\'라고 한단다." "이 그림은 어떠냐 이것도 미술 작품이야. 리엔텐 슈타인은 만화의 한 장면에 검은 선을 그린 다음 알록달록 색칠을 했단다. 진짜 만화책처럼 보이려고 촘촘히 작은 점도 찍었어." "관장님 저도 만화책으로 작품을 만들어 볼래요!" "그리고 리히텐슈타인은 말풍선 안에 글도 넣었단다. 누구나 알기 쉽게 재미있는 그림을 그리고 싶었던 거야. 어때 만화를 보는 것처럼 쉽지?" "네 꼭 만화책 같아요." "햄버거와 막대 아이스크림을 미술 작품이라고 전시한 미

In [18]:
trainset, testset = train_test_split(concat_dataset['동화'], test_size=0.2, shuffle=True, random_state=123)
#trainset, testset

fw = open("C:\\Users\\이육샛별\\Franklin\\KoFairytaleGenerator\\Dataset/trainset_0922.txt", "w", encoding = "utf-8")
fw.write(trainset.str.cat(sep=' ').replace('\n', ' '))
fw.close()

fw = open("C:\\Users\\이육샛별\\Franklin\\KoFairytaleGenerator\\Dataset/testset_0922.txt", "w", encoding = "utf-8")
fw.write(testset.str.cat(sep=' ').replace('\n', ' '))
fw.close()

In [19]:
# from transformers import PreTrainedTokenizerFast
# from transformers import GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM 
import tqdm as notebook_tqdm

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2", use_cache = False,
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

In [21]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', use_cache = False)#.to(device='cuda', non_blocking=True)
# torch.cuda.empty_cache()

In [22]:
prompt = """
지금부터 이야기를 시작하겠습니다. 옛날옛적에 한 오누이가 살고 있었습니다.
"""
with torch.no_grad():
  tokens_bfr = tokenizer.encode(prompt, return_tensors='pt')#.to(device='cuda', non_blocking=True)
  gen_tokens_bfr = model.generate(tokens_bfr, do_sample=True, temperature=0.9, max_length=200, 
                              pad_token_id=tokenizer.pad_token_id,
                              eos_token_id=tokenizer.eos_token_id,
                              bos_token_id=tokenizer.bos_token_id)
  generated_bfr = tokenizer.batch_decode(gen_tokens_bfr)[0].replace("\n", " ")
  
print(generated_bfr)

 지금부터 이야기를 시작하겠습니다. 옛날옛적에 한 오누이가 살고 있었습니다. 어느날, 하루는 그 여인이 어머니 집에 갔다가 우리집의 뜰을 봅니다. 그리고 저는 문틈으로 뭔가 비어져 나오는 것을 찾아봅니다. 마치 어머니가 그녀의 이름을 부르며 문틈으로 파고들기를 기다리는 듯했습니다. 아니, 비어져 나가는 것을 본 순간 어머니는 더 이상 그분을 알지 못합니다. 우리집에 들어오면 그 사람의 이름은 물론 어떤 모습도 보이지 않습니다. 어머니와 저는 바로 그 여인의 얼굴을 다소곳이 쳐다보기도 하면서 그녀의 그 모습을 바라보게 되었습니다. 그러자 그 여인이 "얘, 이 여잘 낳았지?!" 하고 물었습니다. "오늘, 할머니는 괜찮아 보이냐?" 아니, 그 여자는 여자의 이름을 부르며 할머니의 얼굴을 쳐다보았습니다. 여자의 얼굴이 창백해지고 아무것도 보이지 않았습니다. 여자의 얼굴은 창백해진 모습 그대로


In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_path = "/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_trainset.txt"
test_path = "/home/danbi/userdata/SGM_AI/darklady/dataset/tmp_testset.txt"
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [39]:
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [123]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

training_args = TrainingArguments(
    output_dir="/home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0822", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=60, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    logging_steps=20,
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps= 15000, # after # steps model is saved 
    warmup_steps=400,# number of warmup steps for learning rate scheduler
    save_strategy = "steps",
    evaluation_strategy = "steps",
    prediction_loss_only=True,
    optim="adamw_torch",
    dataloader_pin_memory=False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

PyTorch: setting up devices


In [124]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#optimizer.zero_grad()
#with torch.no_grad():
torch.cuda.empty_cache()
trainer.train()

***** Running training *****
  Num examples = 16893
  Num Epochs = 60
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15840


Step,Training Loss,Validation Loss
100,0.210700,5.906689
200,0.230400,5.906446
300,0.243200,5.929402
400,0.281500,5.927177
500,0.305600,5.923293
600,0.277300,5.956725
700,0.296100,5.961024
800,0.284200,5.969594
900,0.274000,5.999148
1000,0.285700,5.999606


***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3557
  Batch size = 64
***** 

In [113]:
trainer.save_model()

Saving model checkpoint to /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819
Configuration saved in /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/config.json
Model weights saved in /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/pytorch_model.bin


In [114]:
finetuned_model_path = "--"
finetuned_model = GPT2LMHeadModel.from_pretrained(finetuned_model_path).to(device='cuda', non_blocking=True)

loading configuration file /home/danbi/userdata/SGM_AI/darklady/kogpt2_finetuned_0819/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_p

In [121]:
#existing code
prompt = """
옛날 옛적에 한 오누이가 살고 있었어요.
"""
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = finetuned_model.generate(tokens, 
                                        early_stopping=True,
                                        do_sample=True,
                                        temperature=0.8, 
                                        max_length=250, 
                                        repetition_penalty=3.0,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        unk_token_id=tokenizer.unk_token_id,)
  generated = tokenizer.decode(gen_tokens[0], skip_special_tokens=True).replace("\n", " ")
  
print(generated)

 옛날 옛적에 한 오누이가 살고 있었어요.  오빠는 너무도 가난하여 양식을 구할 길이 없어 아들아이를 가지길 간절히 바랬고, 자카따룹은 세상을 떠나고 말았어요. 그때 우연히 엄마 나나가 할머니에게 말하는 것을 얻었지요. “할머니, 저하고 결혼하세요.” 사랑하는 아빠를 섬기다 가버린 애가 하루는 꿈에 삼았던 그 새가 살아 있는 걸 보았답니다. 그녀가 창가로 달려가 정말 어느 새 큰 원 안에 갇혀 있게 되었는지 궁금해졌거든요.  그러자 왕비가 이야기를 시작했습니다. “이제 떠돌아다니는 삶에도 지쳤소. 그러니 내가 하느님께 나아갈 수 있도록 도와다오.” 공주가 대답하였어.  “아니,”라며 왕이 자기와 함께 동행한 아기 곰을 데려오셨는데, 그도 마찬가지로 구혼자들이 나타나 이렇게 말하며 그가 원하는 방향으로 헤엄쳐 갔다는 거예요, 그래서 그는 마침내 그를 엄마와 아이를 살려줄 용으로 변신시키고 왕비와 나누어 달라 청하리라 부르게 된 거지 뭐죠. 하지만 이 일로 왕비는 납북절한테는 어떤 약속 말고 그냥 묻어주시기만 하면 된다는 뜻이었습니다.  넌 분명 아름답고 영리한테서 그리고 아무 것도 얻을 수가 있는데 말이 있어 그리할 것이며, 그렇다면 그렇게 말하고 싶진 않으냐 그러면서
